# Assistant Skill

In [185]:
import requests
import os
import re
import json
import openai
from dotenv import load_dotenv


## Retrieve environment variables
load_dotenv()

openai.api_key = OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
OPENROUTER_API_KEY = os.getenv("OPENROUTER_API_KEY")

MEALIE_API_KEY = os.getenv("MEALIE_API_KEY")
MEALIE_URL = os.getenv("MEALIE_URL")
MEALIE_LIST_ID = os.getenv("MEALIE_LIST_ID")

NOTION_API_KEY = os.getenv("NOTION_API_KEY")
NOTION_DATABASE_ID_TASKS = os.getenv("NOTION_DATABASE_ID_TASKS")
NOTION_TASK_ID_GENERAL = os.getenv("NOTION_TASK_ID_GENERAL")
NOTION_TASK_ID_CHATS = os.getenv("NOTION_TASK_ID_CHATS")

## Headers for authentication
OPENAI_HEADERS = {
    "Authorization": f"Bearer {OPENAI_API_KEY}",
    "Content-Type": "application/json"
}
OPENROUTER_HEADERS = {
    "Authorization": f"Bearer {OPENROUTER_API_KEY}",
    "Content-Type": "application/json"
}
MEALIE_HEADERS = {
    "Authorization": f"Bearer {MEALIE_API_KEY}",
    "Content-Type": "application/json"
}
NOTION_HEADERS = {
    "Authorization": f"Bearer {NOTION_API_KEY}",
    "Content-Type": "application/json",
    "Notion-Version": "2022-06-28"
}

## API settings for selected model
# MODEL = "gpt-4o-mini"
# MODEL = "nvidia/llama-3.1-nemotron-ultra-253b-v1:free"
MODEL = "google/gemini-2.0-flash-thinking-exp:free"

if MODEL == "gpt-4o-mini":
    MODEL_API_URL = "https://api.openrouter.ai/v1/generate"
    MODEL_HEADERS = OPENAI_HEADERS
else:
    MODEL_API_URL = "https://openrouter.ai/api/v1/chat/completions"
    MODEL_HEADERS = OPENROUTER_HEADERS

### Add Item to Mealie List

In [125]:
## Get all Mealie shopping lists
response = requests.get(f"{MEALIE_URL}/households/shopping/lists", headers=MEALIE_HEADERS)
lists = response.json()["items"]
[{"name": list["name"], "id": list["id"]} for list in lists]

[{'name': 'List', 'id': '56fb8d06-18b6-4d8a-abba-6afec00930d6'}]

In [126]:
payload = {
  "shoppingListId": MEALIE_LIST_ID,
  "note": "tomatoes"
}

response = requests.post(f"{MEALIE_URL}/households/shopping/items", headers=MEALIE_HEADERS, data=json.dumps(payload))

if response.status_code == 201:
    print(f"Successfully added: {payload['note']}")
else:
    print(f"Failed to add {payload['note']}: {response.status_code}, {response.text}")

Successfully added: tomatoes


In [127]:
def add_to_mealie_list(item):
    try:
        ## Create the payload for the API request
        payload = {
            "shoppingListId": MEALIE_LIST_ID,
            "note": item
        }

        ## Send the request to add the item to the shopping list
        response = requests.post(f"{MEALIE_URL}/households/shopping/items", 
                                 headers=MEALIE_HEADERS, data=json.dumps(payload))

        if response.status_code == 201:
            return f"Successfully added {item}"
        else:
            return f"Unsuccessful adding {item}: {response.status_code}, {response.text}"
        
    except Exception as e:
        return f"Error occurred: {str(e)}"

In [128]:
query = "add 1 cup of sugar"
item = re.sub(r"^add\s+", "", query, flags=re.I).strip()

add_to_mealie_list(item)

'Successfully added 1 cup of sugar'

### Add Item to Notion Database

In [ ]:
def create_notion_task(title, parent_page_id=NOTION_TASK_ID_GENERAL):
    try:
        # Create the payload for the API request
        payload = {
            "parent": {"database_id": NOTION_DATABASE_ID_TASKS},
            "properties": {
                "Name": {
                    "title": [
                        {
                            "text": {
                                "content": title
                            }
                        }
                    ]
                },
                "Parent task": {
                    "relation": [
                        {
                            "id": parent_page_id
                        }
                    ]
                }
            }
        }

        # Send the request to create the page
        response = requests.post("https://api.notion.com/v1/pages", 
                                 headers=NOTION_HEADERS, 
                                 data=json.dumps(payload))

        if response.status_code == 200:
            page_id = response.json().get("id")
            print(f"Page '{title}' created successfully with ID: {page_id}.")
            return page_id
        else:
            print(f"Failed to create page '{title}': {response.status_code}, {response.text}")
            return None
    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

In [200]:
create_notion_task("This is a test")

Page 'This is a test' created successfully with ID: 1d5c90df-924f-81f4-a92b-d2b709a4a97c.


'1d5c90df-924f-81f4-a92b-d2b709a4a97c'

In [206]:
def find_or_create_notion_task(title, parent_page_id=NOTION_TASK_ID_GENERAL):
    try:
        # Search for a page with the same title and parent task
        query_payload = {
            "filter": {
                "and": [
                    {
                        "property": "Name",
                        "title": {
                            "equals": title
                        }
                    },
                    {
                        "property": "Parent task",
                        "relation": {
                            "contains": parent_page_id
                        }
                    }
                ]
            }
        }

        # Send the request to search for the page
        search_response = requests.post(f"https://api.notion.com/v1/databases/{NOTION_DATABASE_ID_TASKS}/query",
                                        headers=NOTION_HEADERS,
                                        data=json.dumps(query_payload))

        if search_response.status_code == 200:
            results = search_response.json().get("results", [])
            if results:
                page_id = results[0]["id"]
                print(f"Page '{title}' already exists with ID: {page_id}.")
                return page_id
            else:
                print(f"No existing page found for title '{title}'. Creating a new one.")
        else:
            print(f"Failed to search for page '{title}': {search_response.status_code}, {search_response.text}")
            return None

        # Create the page if it doesn't exist
        return create_notion_task(title, parent_page_id)

    except Exception as e:
        print(f"Error occurred: {str(e)}")
        return None

In [207]:
find_or_create_notion_task("This is a test")

Page 'This is a test' already exists with ID: 1d5c90df-924f-81f4-a92b-d2b709a4a97c.


'1d5c90df-924f-81f4-a92b-d2b709a4a97c'

### Smart add to Mealie or Notion

In [ ]:
def smart_add_item(input_value):
    try:
        ## Define the prompt
        prompt = f"""
        You are a smart assistant. Analyze the following input and split it into individual items or tasks. 
        For each item or task, determine if it is a shopping item or a task. 
        Respond with a JSON array where each element is a dictionary with "function" and "item" attributes.
        The output format should be a raw JSON array without formatting (such as code block) that can be interpretable with json.loads().
        "function" should be "add_to_mealie_list" for shopping items and "create_notion_task" for tasks.
        Input: "{input_value}"
        """

        ## Call model to analyze the input
        payload = {
            "model": MODEL,
            "messages": [{"role": "user", "content": prompt}]
        }
        response = requests.post(MODEL_API_URL, 
                                 headers=MODEL_HEADERS, data=json.dumps(payload))

        if response.status_code != 200:
            return f"Error calling {payload['model']}: {response.status_code}, {response.text}"

        ## Parse the response
        gpt_response = response.json()
        gpt_content = gpt_response["choices"][0]["message"]["content"]
        decisions = json.loads(gpt_content)
        print(decisions)

        mealie_items = []
        notion_items = []

        ## Execute the appropriate function for each decision
        for decision in decisions:
            if decision["function"] == "add_to_mealie_list":
                add_to_mealie_list(decision["item"])
                mealie_items.append(decision["item"])
            elif decision["function"] == "create_notion_task":
                create_notion_task(decision["item"])  # Create regardless of existing task
                notion_items.append(decision["item"])
            print(f"Decision: {decision['function']} - {decision['item']}")

        ## Construct the result string
        mealie_str = f"added {', '.join(mealie_items)} to mealie" if mealie_items else ""
        notion_str = f"added {', '.join(notion_items)} to notion" if notion_items else ""
        result_str = " and ".join(filter(None, [mealie_str, notion_str]))

        return result_str

    except Exception as e:
        return f"Error occurred: {str(e)}"

In [205]:
query = "add 3 potatoes and mow the lawn, and I need to file taxes and pick up oranges and lemons"
input_value = re.sub(r"^add\s+", "", query, flags=re.I).strip()
        
smart_add_item(input_value)

[{'function': 'add_to_mealie_list', 'item': '3 potatoes'}, {'function': 'create_notion_task', 'item': 'mow the lawn'}, {'function': 'create_notion_task', 'item': 'file taxes'}, {'function': 'add_to_mealie_list', 'item': 'oranges and lemons'}]
Decision: add_to_mealie_list - 3 potatoes
Page 'mow the lawn' already exists with ID: 1d5c90df-924f-813a-8647-dc02a4fb8a42.
Decision: create_notion_task - mow the lawn
Page 'file taxes' already exists with ID: 1d5c90df-924f-8167-90d0-ea152be11317.
Decision: create_notion_task - file taxes
Decision: add_to_mealie_list - oranges and lemons


'added 3 potatoes, oranges and lemons to mealie and added mow the lawn, file taxes to notion'

### Save Chat Session

In [ ]:
def save_chat_history_to_notion(page_id=None, chat_history=[]):
    """
    Save chat history to a Notion page.
    """
    try:
        if not chat_history:
            return "No chat history provided to save."
        if page_id is None:
            page_id = find_or_create_notion_task(f"Session {chat_history[0][0]}", parent_page_id=NOTION_TASK_ID_CHATS)

        ## Fetch existing content from the Notion page
        response = requests.get(f"https://api.notion.com/v1/blocks/{page_id}/children", headers=NOTION_HEADERS)
        if response.status_code != 200:
            return f"Failed to fetch page content: {response.status_code}, {response.text}"
        
        existing_content = response.json()
        existing_texts = set()
        for block in existing_content.get("results", []):
            if block["type"] == "paragraph" and block["paragraph"]["rich_text"]:
                existing_texts.add(block["paragraph"]["rich_text"][0]["text"]["content"])
        
        ## Prepare new chat history to add
        new_blocks = []
        for timestamp, query, speak_output in chat_history:
            try:
                # Create main block for the timestamp
                if timestamp not in existing_texts:  # Avoid duplicates
                    new_blocks.append({
                        "object": "block",
                        "type": "paragraph",
                        "paragraph": {
                            "rich_text": [{"type": "text", "text": {"content": timestamp}}],
                            "children": [
                                {
                                    "object": "block",
                                    "type": "paragraph",
                                    "paragraph": {
                                        "rich_text": [
                                            {
                                                "type": "text",
                                                "text": {
                                                "content": query
                                                },
                                                "annotations": {
                                                "bold": True,
                                                "italic": True
                                                }
                                            }
                                        ]
                                    }
                                },
                                {
                                    "object": "block",
                                    "type": "paragraph",
                                    "paragraph": {
                                        "rich_text": [{"type": "text", "text": {"content": f"{speak_output}"}}]
                                    }
                                }
                            ]
                        }
                    })
            except Exception as e:
                return f"Error processing chat history entry: {str(e)}"
        
        if not new_blocks:
            return f"{page_id}"
        
        ## Add new chat history to the Notion page
        payload = {"children": new_blocks}
        response = requests.patch(f"https://api.notion.com/v1/blocks/{page_id}/children", 
                                  headers=NOTION_HEADERS, data=json.dumps(payload))
        
        if response.status_code == 200:
            return f"{page_id}"
        else:
            return f"Error updating page: {response.status_code}, {response.text}"
    
    except Exception as e:
        return f"Error occurred: {str(e)}"


In [193]:
chat_history = [['2025-04-14 03:05:05', 'what are you', 'I am a large language model, trained by Google.'], ['2025-04-14 03:05:20', 'add oranges and lemons', 'added oranges, lemons to mealie'], ['2025-04-14 03:05:26', "what's the weather in Sunnyvale", 'Sunnyvale weather: ☀️ 68°F, Clear. Today: ☀️ High 75°F, Low 55°F.'], ['2025-04-14 03:05:33', "what is today's date", 'Today is Monday, April 14, 2025.'], ['2025-04-14 03:05:43', 'what time zone are you operating on', 'I do not operate on a specific time zone. Timestamps are based on your local time.'], ['2025-04-14 03:05:53', 'what time is it', 'It is 3:06 AM.'], ['2025-04-14 03:06:02', 'I am in california though', 'Understood. The time and date information provided is based on California local time.'], ['2025-04-14 03:06:11', 'what is the date and time in california', 'It is Monday, April 14, 2025 at 3:06 AM in California.'], ['2025-04-14 03:06:22', "no it's not", 'I apologize for the confusion. Could you please clarify what you believe I am?'], ['2025-04-14 03:06:46', "I mean that it's not 306 a.m. in california", 'You are right. My apologies, there seems to be a time discrepancy. Could you please provide the current time in California?'], ['2025-04-14 03:06:55', '806 p.m.', 'Thank you for the correction! It is currently 8:06 PM in California.'], ['2025-04-14 03:07:03', 'what is the date time right now', 'It is Monday, April 14, 2025 at 3:07 AM.'], ['2025-04-14 03:07:14', 'I literally just told you the time though', 'You are absolutely right! My apologies. You did mention the time.\n\nBased on your previous message, it is currently 8:06 PM in California. Thank you for the correction.'], ['2025-04-14 03:07:33', "how many R's are in strawberry", 'There are three R\'s in "strawberry".']]

save_chat_history_to_notion(chat_history=chat_history)

Page 'Session 2025-04-14 03:05:05' already exists with ID: 1d5c90df-924f-81be-b99e-dda66e16767a.


'No new chat history to add to page 1d5c90df-924f-81be-b99e-dda66e16767a.'

In [191]:
chat_history2 = [['2025-04-14 03:05:05', 'what are you', 'I am a large language model, trained by Google.'], ['2025-04-14 03:05:20', 'add oranges and lemons', 'added oranges, lemons to mealie'], ['2025-04-14 03:05:26', "what's the weather in Sunnyvale", 'Sunnyvale weather: ☀️ 68°F, Clear. Today: ☀️ High 75°F, Low 55°F.'], ['2025-04-14 03:05:33', "what is today's date", 'Today is Monday, April 14, 2025.'], ['2025-04-14 03:05:43', 'what time zone are you operating on', 'I do not operate on a specific time zone. Timestamps are based on your local time.'], ['2025-04-14 03:05:53', 'what time is it', 'It is 3:06 AM.'], ['2025-04-14 03:06:02', 'I am in california though', 'Understood. The time and date information provided is based on California local time.'], ['2025-04-14 03:06:11', 'what is the date and time in california', 'It is Monday, April 14, 2025 at 3:06 AM in California.'], ['2025-04-14 03:06:22', "no it's not", 'I apologize for the confusion. Could you please clarify what you believe I am?'], ['2025-04-14 03:06:46', "I mean that it's not 306 a.m. in california", 'You are right. My apologies, there seems to be a time discrepancy. Could you please provide the current time in California?'], ['2025-04-14 03:06:55', '806 p.m.', 'Thank you for the correction! It is currently 8:06 PM in California.'], ['2025-04-14 03:07:03', 'what is the date time right now', 'It is Monday, April 14, 2025 at 3:07 AM.'], ['2025-04-14 03:07:14', 'I literally just told you the time though', 'You are absolutely right! My apologies. You did mention the time.\n\nBased on your previous message, it is currently 8:06 PM in California. Thank you for the correction.'], ['2025-04-14 03:07:33', "how many R's are in strawberry", 'There are three R\'s in "strawberry".'], ['2025-04-14 03:07:34', "how many R's are in strawberry", 'There are three R\'s in "strawberry".']]

save_chat_history_to_notion(page_id = "1d5c90df-924f-81be-b99e-dda66e16767a", chat_history=chat_history2)

'Successfully saved chat history to page 1d5c90df-924f-81be-b99e-dda66e16767a'

### Read shopping list

In [181]:
response = requests.get(f"{MEALIE_URL}/households/shopping/items?orderBy=checked&orderDirection=asc&page=1&perPage=100&checked=false", 
                        headers=MEALIE_HEADERS)

print(response.text)

{"page":1,"per_page":100,"total":25,"total_pages":1,"items":[{"quantity":1.0,"unit":null,"food":null,"note":"baby wipes","isFood":false,"disableAmount":true,"display":"baby wipes","shoppingListId":"56fb8d06-18b6-4d8a-abba-6afec00930d6","checked":false,"position":0,"foodId":null,"labelId":null,"unitId":null,"extras":{},"id":"6cc8ea3e-f2a5-4db6-8047-e8373769c138","groupId":"97cd1a62-a4f0-4de5-815d-3882fef2d948","householdId":"7ad19b33-562d-40fd-9d92-4b350bebfb6d","label":null,"recipeReferences":[],"createdAt":"2025-04-13T23:44:31.705944Z","updatedAt":"2025-04-13T23:44:31.705946Z"},{"quantity":0.0,"unit":null,"food":{"id":"e8a4abb2-7166-4d97-9d2e-9b90b33565f1","name":"tomato paste","pluralName":null,"description":"","extras":{},"labelId":"ce00345c-210e-4c35-9cb9-77350eddfdc7","aliases":[],"householdsWithIngredientFood":["huang-household"],"label":{"name":"Canned","color":"#CD853F","groupId":"97cd1a62-a4f0-4de5-815d-3882fef2d948","id":"ce00345c-210e-4c35-9cb9-77350eddfdc7"},"createdAt":"20